# Master's thesis code

**Sofia Morais - 152021001**

In [1]:
# Load the packages
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

## 1. DATA PREPARATION

In [28]:
# Import the original dataset and save it
original = pd.read_excel('C:/Users/sofia/OneDrive/Desktop/CLSBE/2nd year/Tese/data/original.xlsx', header=1,index_col=0)

# Data transformation
original.index.name='Timestamp'
original = original.rename(columns={'google': 'btc_popularity'})
original = original.rename(columns={'google.1': 'btc_popularity.1'})
original['Confidence']= ''
original
original.head()

,market-price,market-cap,trade-volume,transaction-fees-usd,avg-confirmation-time,difficulty,High,Low,hash-rate,block-size,...,btc_popularity.1,Open.1,n-payments-per-block.1,n-transactions-per-block.2,cost-per-transaction-percent.1,fees-usd-per-transaction.1,n-unique-addresses.1,n-transactions-per-block.3,output-volume.1,Confidence
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1013.42,1.651285e+10,4.765228e+06,114872.988609,13.016229,3.176884e+11,1013.42,956.10,2.589950e+06,97293.875547,...,0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,
2017-01-02,1013.42,1.651285e+10,4.765228e+06,114872.988609,97.413264,3.176884e+11,1031.70,990.20,2.589950e+06,97293.875547,...,0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,
2017-01-03,1013.42,1.651285e+10,4.765228e+06,114872.988609,16.441754,3.176884e+11,1035.50,1006.50,2.589950e+06,97293.875547,...,31,32.0,0.0,0.000000,-0.031183,0.00000,0.0,0.000000,0.0000,
2017-01-04,999.65,1.758021e+10,5.857834e+06,114872.988609,37.127815,3.176884e+11,1148.50,999.65,2.542573e+06,97743.656720,...,-5,21.6,0.0,0.000000,-0.007974,0.00000,0.0,0.000000,0.0000,
2017-01-05,999.65,1.758021e+10,5.857834e+06,121958.044759,17.147923,3.176884e+11,1150.60,874.50,2.147763e+06,97743.656720,...,10,16.3,0.0,21.806066,-0.050283,0.01984,19705.8,3653.933333,104726.6494,


In [43]:
# Import the confidence index dataset and save it
confidence = pd.read_csv('C:/Users/sofia/OneDrive/Desktop/CLSBE/2nd year/Tese/data/consumer confidence index.csv',index_col='TIME')

# Data transformation
confidence= confidence[confidence["LOCATION"].str.contains("OECD")==True]
confidence= confidence[confidence["LOCATION"].str.contains("OECDE")==False]
confidence= confidence.drop(['LOCATION','INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes'], axis='columns')
# Delete the the rows that you will not use

for i in confidence.index:
    if i>'2021-06':
        confidence=confidence.drop(i)
    continue

# Check the last values to make sure that you the previous code worked
confidence.head()

,Value
TIME,
2017-01,100.5115
2017-02,100.5401
2017-03,100.6480
2017-04,100.7638
2017-05,100.8411


In [42]:
crows = len(confidence.index) # number of rows in the confidence dataset
orows = len(original.index)  # number of rows in the original dataset

# Add the confidence index variable to the original dataset
for i in original.index.strftime('%Y-%m'):
    for j in confidence.index:
        if i==j:
            original['Confidence'][i]=confidence['Value'][j]
    continue
print(original['Confidence'].head(64))

#original['Confidence'][i] = confidence[data['City']=='Vienna'].copy()

<ipython-input-42-6274d6e39ff2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Confidence'][i]=confidence['Value'][j]


Timestamp
2017-01-01    100.5115
2017-01-02    100.5115
2017-01-03    100.5115
2017-01-04    100.5115
2017-01-05    100.5115
                ...   
2017-03-01     100.648
2017-03-02     100.648
2017-03-03     100.648
2017-03-04     100.648
2017-03-05     100.648
Name: Confidence, Length: 64, dtype: object


In [34]:
# Import the binary variable of bitcoin regulation related news and save it
news_reg = pd.read_excel('C:/Users/sofia/OneDrive/Desktop/CLSBE/2nd year/Tese/data/news_regulation.xlsx', header=0,index_col=0)
news_reg.head()

# Add this variable to the original dataset
original['News Regulation'] = news_reg['news_regulation']
original.head() #check the original dataset

,market-price,market-cap,trade-volume,transaction-fees-usd,avg-confirmation-time,difficulty,High,Low,hash-rate,block-size,...,Open.1,n-payments-per-block.1,n-transactions-per-block.2,cost-per-transaction-percent.1,fees-usd-per-transaction.1,n-unique-addresses.1,n-transactions-per-block.3,output-volume.1,Confidence,News Regulation
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1013.42,1.651285e+10,4.765228e+06,114872.988609,13.016229,3.176884e+11,1013.42,956.10,2.589950e+06,97293.875547,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,100.5115,0
2017-01-02,1013.42,1.651285e+10,4.765228e+06,114872.988609,97.413264,3.176884e+11,1031.70,990.20,2.589950e+06,97293.875547,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0000,100.5115,0
2017-01-03,1013.42,1.651285e+10,4.765228e+06,114872.988609,16.441754,3.176884e+11,1035.50,1006.50,2.589950e+06,97293.875547,...,32.0,0.0,0.000000,-0.031183,0.00000,0.0,0.000000,0.0000,100.5115,0
2017-01-04,999.65,1.758021e+10,5.857834e+06,114872.988609,37.127815,3.176884e+11,1148.50,999.65,2.542573e+06,97743.656720,...,21.6,0.0,0.000000,-0.007974,0.00000,0.0,0.000000,0.0000,100.5115,0
2017-01-05,999.65,1.758021e+10,5.857834e+06,121958.044759,17.147923,3.176884e+11,1150.60,874.50,2.147763e+06,97743.656720,...,16.3,0.0,21.806066,-0.050283,0.01984,19705.8,3653.933333,104726.6494,100.5115,0


In [ ]:
## Sentiment analysis using Google Trends (GT)
# Import the variables
GT = pd.read_csv('C:/Users/sofia/OneDrive/Desktop/CLSBE/2nd year/Tese/data/GT.csv', header=1,index_col=0)

#Rename the variables to simplify
GT = GT.rename(columns={'bitcoin legal: (Worldwide)': 'legal'})
GT = GT.rename(columns={'bitcoin cryptocurrency: (Worldwide)': 'cryptocurrency'})
GT = GT.rename(columns={'bitcoin scam: (Worldwide)': 'scam'})
GT.head()

In [ ]:
# no dataset final: clean the data

## 2. DATA EXPLORATION